In [2]:
import numpy as np
import pandas as pd
import os
import sys
import random
import keras
import tensorflow as tf
import json
import bert
from bert import tokenization

Using TensorFlow backend.
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

In [3]:
from keras_bert.bert import get_model
from keras_bert.loader import load_trained_model_from_checkpoint
from keras.optimizers import Adam
adam = Adam(lr=2e-5,decay=0.01)
maxlen = 75
BERT_PRETRAINED_DIR = '.../'  ##-- point to source directory after placing the 'BERT-Base Uncased' files downloaded from "https://github.com/google-research/bert"
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)

In [4]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
import codecs

sequence_output  = model.layers[-6].output
pool_output = Dense(5, activation='softmax',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'real_output')(sequence_output)
model3 = Model(inputs=model.input, outputs=pool_output)
model3.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

In [5]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for i in range(example.shape[0]):
      tokens_a = tokenizer.tokenize(example[i])
      if len(tokens_a)>max_seq_length:
        tokens_a = tokens_a[:max_seq_length]
        longer += 1
      one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
      all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [16]:
dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
def create_tokenizer(vocab_file, do_lower_case=False):
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)
tokenizer = create_tokenizer(dict_path, do_lower_case=False)
print('build tokenizer done')

build tokenizer done


In [17]:
model3.load_weights('D:/Azure_Batch2_Recordings/bert_weights_final.h5') ##-- place and point to bert_weights_final.h5 in the source folder

In [18]:
df_input = pd.read_excel('D:/Nils_18_feb/val26.xlsx') ##-- point to input file, which is to be predicted

In [19]:
df_input['contents'] = df_input['contents'].str.lower()
df_pred = df_input['contents'].dropna()
eval_lines = df_pred.values

In [20]:
token_input2 = convert_lines(eval_lines,maxlen,tokenizer)
seg_input2 = np.zeros((token_input2.shape[0],maxlen))
mask_input2 = np.ones((token_input2.shape[0],maxlen))
emotions = model3.predict([token_input2, seg_input2, mask_input2],verbose=1,batch_size=bsz)

W0226 17:54:41.772096 38620 deprecation_wrapper.py:119] From c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



1
23/23 [==============================] - 5s 219ms/step


In [21]:
df_prediction = pd.DataFrame(df_pred)

In [22]:
labels = np.argmax(emotions, axis=-1)
emotion_class = {0: "frustrated", 1: "disappointed", 2: "neutral", 3: "happy", 4: "pleased"}

In [23]:
df_prediction['emotions'] = labels.tolist()

In [24]:
df_prediction=df_prediction.replace({"emotions": emotion_class})

In [25]:
df_prediction.to_csv('pred_output.csv')